In [1]:
from sentence_transformers import SentenceTransformer
from pymongo import MongoClient, ASCENDING, DESCENDING, HASHED
from tqdm.notebook import tqdm
import torch

def set_device():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device != "cuda":
        print("WARNING: For this notebook to perform best, "
              "if possible, in the menu under `Runtime` -> "
              "`Change runtime type.`  select `GPU` ")
    else:
        print("GPU is enabled in this notebook.")
    return device


device = set_device()


conn = MongoClient()

fake_fibvid = conn.fake_fibvid
tweets  = fake_fibvid.tweets

process_fibvid = conn.process_fibvid
tweets_st  = process_fibvid.tweets_st

tweets_st.create_index([('created', ASCENDING)])
tweets_st.create_index([('tweetId', HASHED)])
tweets_st.create_index([('userId', HASHED)])

model = SentenceTransformer('all-mpnet-base-v2').to(device)

GPU is enabled in this notebook.


In [2]:
def process(to_process):
    to_store = []
    texts = [tweet['text'] for tweet in to_process]
    embs = model.encode(texts)
    for tweet, emb in zip(to_process, embs):
        store = {}
        store['embs'] = emb.tolist()
        store['created'] = tweet['created']
        store['userId'] = tweet['userId']
        store['tweetId'] = tweet['tweetId']
        to_store.append(store)
    tweets_st.insert_many(to_store)

In [3]:
total = tweets.count_documents({})

to_process = []

for tweet in tqdm(tweets.find(), total=total):
    to_process.append(tweet)
    if len(to_process) > 10000:
        process(to_process)
        to_process = []
        

if len(to_process) > 0:
    process(to_process)
    to_process = []

  0%|          | 0/299118 [00:00<?, ?it/s]